# Understanding how the SQuAD dataset is set up for our task with BERT

We are going to fine-tune [BERT implemented by HuggingFace](https://huggingface.co/bert-base-uncased) for the text-extraction task with a dataset of questions and answers with the [SQuAD (The Stanford Question Answering Dataset)](https://rajpurkar.github.io/SQuAD-explorer/) dataset.
The data is composed by a set of questions and corresponding paragraphs that contains the answers.
The model will be trained to locate the answer in the context by giving the positions where the answer starts and ends.

In this notebook we are going to see how the data is set up for training.

This notebook is based on [BERT (from HuggingFace Transformers) for Text Extraction](https://keras.io/examples/nlp/text_extraction_with_bert/).

More info:
- [Glossary - HuggingFace docs](https://huggingface.co/transformers/glossary.html#model-inputs)
- [BERT NLP — How To Build a Question Answering Bot](https://towardsdatascience.com/bert-nlp-how-to-build-a-question-answering-bot-98b1d1594d7b)

In [1]:
import os
import json
import dataset_utils as du
from transformers import BertTokenizer
from tokenizers import BertWordPieceTokenizer

## The raw data

In [2]:
bert_cache = os.path.join(os.getcwd(), 'cache')

train_path = os.path.join(bert_cache, 'data', 'train-v1.1.json')
eval_path = os.path.join(bert_cache, 'data', 'dev-v1.1.json')
with open(train_path) as f:
    raw_train_data = json.load(f)

with open(eval_path) as f:
    raw_eval_data = json.load(f)

In [3]:
raw_train_data.keys()

dict_keys(['data', 'version'])

In [4]:
print(f"- {len(raw_train_data['data'])} contexts for training")
print(f"- {len(raw_eval_data['data'])} contexts for testing")

- 442 contexts for training
- 48 contexts for testing


In [5]:
raw_train_data['data'][91]['title']

'Alps'

In [6]:
len(raw_train_data['data'][91]['paragraphs'])

87

In [7]:
raw_train_data['data'][91]['paragraphs'][0]['context']

'The Alps (/ælps/; Italian: Alpi [ˈalpi]; French: Alpes [alp]; German: Alpen [ˈʔalpm̩]; Slovene: Alpe [ˈáːlpɛ]) are the highest and most extensive mountain range system that lies entirely in Europe, stretching approximately 1,200 kilometres (750 mi) across eight Alpine countries: Austria, France, Germany, Italy, Liechtenstein, Monaco, Slovenia, and Switzerland. The Caucasus Mountains are higher, and the Urals longer, but both lie partly in Asia. The mountains were formed over tens of millions of years as the African and Eurasian tectonic plates collided. Extreme shortening caused by the event resulted in marine sedimentary rocks rising by thrusting and folding into high mountain peaks such as Mont Blanc and the Matterhorn. Mont Blanc spans the French–Italian border, and at 4,810 m (15,781 ft) is the highest mountain in the Alps. The Alpine region area contains about a hundred peaks higher than 4,000 m (13,123 ft), known as the "four-thousanders".'

In [8]:
raw_train_data['data'][91]['paragraphs'][0]['qas']

[{'answers': [{'answer_start': 190, 'text': 'Europe'}],
  'question': 'What Country are the Alps located in?',
  'id': '56f81f0ea6d7ea1400e173d7'},
 {'answers': [{'answer_start': 223, 'text': '1,200 kilometres'}],
  'question': 'How many kilometres do the Alps stretch?',
  'id': '56f81f0ea6d7ea1400e173d8'},
 {'answers': [{'answer_start': 475, 'text': 'over tens of millions of years'}],
  'question': 'How long has it taken for the Alps to form? ',
  'id': '56f81f0ea6d7ea1400e173d9'},
 {'answers': [{'answer_start': 732, 'text': 'Mont Blanc'}],
  'question': 'What is the highest mountain in the Alps?',
  'id': '56f81f0ea6d7ea1400e173da'},
 {'answers': [{'answer_start': 936, 'text': 'the "four-thousanders"'}],
  'question': 'The Alpine region is also known as what? ',
  'id': '56f81f0ea6d7ea1400e173db'}]

## The tokenizer

In [9]:
hf_model = 'bert-base-uncased'

In [10]:
slow_tokenizer = BertTokenizer.from_pretrained(
    hf_model,
    cache_dir=os.path.join(bert_cache, f'_{hf_model}-tokenizer')
)
save_path = os.path.join(bert_cache, f'{hf_model}-tokenizer')
if not os.path.exists(save_path):
    os.makedirs(save_path)
    slow_tokenizer.save_pretrained(save_path)
    
# Load the fast tokenizer from saved file
tokenizer = BertWordPieceTokenizer(os.path.join(save_path, 'vocab.txt'),
                                   lowercase=True)

## Processing the data for training
Now we process the data so we can feed it later to the model.
The idea is to replace the words (and some word parts) by numbers using the tokenizer above and organize the training data as a set of paragraphs and questions.

In [11]:
%%time

max_len = 384

train_squad_examples = du.create_squad_examples(raw_train_data, max_len, tokenizer)
x_train, y_train = du.create_inputs_targets(train_squad_examples)
print(f"{len(train_squad_examples)} training points created.")

86136 training points created.
CPU times: user 1min 1s, sys: 1.19 s, total: 1min 2s
Wall time: 1min 2s


### The SquadExample objects

In [12]:
type(train_squad_examples)

numpy.ndarray

In [13]:
train_squad_examples.shape

(86136,)

In [14]:
sample = 20299  # take the sample number 20299 out of the 86136 samples

train_squad_examples[sample]

In [15]:
train_squad_examples[sample].__dict__.keys()

dict_keys(['question', 'context', 'start_char_idx', 'answer_text', 'all_answers', 'max_len', 'skip', 'tokenizer', 'input_ids', 'token_type_ids', 'attention_mask', 'start_token_idx', 'end_token_idx', 'context_token_to_char'])

### The training set

In [16]:
len(x_train)

3

In [17]:
x_train[0].shape, x_train[1].shape, x_train[2].shape

((86136, 384), (86136, 384), (86136, 384))

In [18]:
x_train[0][sample]

array([  101,  1996, 13698,  1006,  1013,  1097, 14277,  2015,  1013,
        1025,  3059,  1024,  2632,  8197,  1031,  1149,  2389,  8197,
        1033,  1025,  2413,  1024,  2632, 10374,  1031,  2632,  2361,
        1033,  1025,  2446,  1024,  2632, 11837,  1031,  1149, 29705,
        2389,  9737,  1033,  1025, 18326,  1024,  2632,  5051,  1031,
        1149,  2050, 23432, 14277, 29275,  1033,  1007,  2024,  1996,
        3284,  1998,  2087,  4866,  3137,  2846,  2291,  2008,  3658,
        4498,  1999,  2885,  1010, 10917,  3155,  1015,  1010,  3263,
        3717,  1006,  9683,  2771,  1007,  2408,  2809, 10348,  3032,
        1024,  5118,  1010,  2605,  1010,  2762,  1010,  3304,  1010,
       26500,  1010, 14497,  1010, 10307,  1010,  1998,  5288,  1012,
        1996, 16512,  4020,  2024,  3020,  1010,  1998,  1996, 24471,
        9777,  2936,  1010,  2021,  2119,  4682,  6576,  1999,  4021,
        1012,  1996,  4020,  2020,  2719,  2058, 15295,  1997,  8817,
        1997,  2086,

In [19]:
# A training sample is a paragraph plus a question
#
# The padding zeroes are discarded by the tokenizer's decoding
tokenizer.decode(x_train[0][sample])

'the alps ( / ælps / ; italian : alpi [ ˈalpi ] ; french : alpes [ alp ] ; german : alpen [ ˈʔalpm ] ; slovene : alpe [ ˈaːlpɛ ] ) are the highest and most extensive mountain range system that lies entirely in europe, stretching approximately 1, 200 kilometres ( 750 mi ) across eight alpine countries : austria, france, germany, italy, liechtenstein, monaco, slovenia, and switzerland. the caucasus mountains are higher, and the urals longer, but both lie partly in asia. the mountains were formed over tens of millions of years as the african and eurasian tectonic plates collided. extreme shortening caused by the event resulted in marine sedimentary rocks rising by thrusting and folding into high mountain peaks such as mont blanc and the matterhorn. mont blanc spans the french – italian border, and at 4, 810 m ( 15, 781 ft ) is the highest mountain in the alps. the alpine region area contains about a hundred peaks higher than 4, 000 m ( 13, 123 ft ), known as the " four - thousanders ". ho

## [Token type ids](https://huggingface.co/transformers/glossary.html#token-type-ids)

In [20]:
# `x_train[1][i]` is 1 for the elements that represent the question:
x_train[1][sample]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [21]:
# `x_train[1][i]==1`, the question:
question_encoded = x_train[0][sample][x_train[1][sample] == 1]
tokenizer.decode(question_encoded)

'how long has it taken for the alps to form?'

## [Attention masks](https://huggingface.co/transformers/glossary.html#attention-mask)

In [22]:
# `x_train[2][i]` has the value 1 for the elements that represent the text:
x_train[2][sample]

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [23]:
# `x_train[1][i]==0` selects the context and `x_train[1][i]==1`, the question:
context_encoded = x_train[0][sample][x_train[2][sample] == 1]
tokenizer.decode(context_encoded)

'the alps ( / ælps / ; italian : alpi [ ˈalpi ] ; french : alpes [ alp ] ; german : alpen [ ˈʔalpm ] ; slovene : alpe [ ˈaːlpɛ ] ) are the highest and most extensive mountain range system that lies entirely in europe, stretching approximately 1, 200 kilometres ( 750 mi ) across eight alpine countries : austria, france, germany, italy, liechtenstein, monaco, slovenia, and switzerland. the caucasus mountains are higher, and the urals longer, but both lie partly in asia. the mountains were formed over tens of millions of years as the african and eurasian tectonic plates collided. extreme shortening caused by the event resulted in marine sedimentary rocks rising by thrusting and folding into high mountain peaks such as mont blanc and the matterhorn. mont blanc spans the french – italian border, and at 4, 810 m ( 15, 781 ft ) is the highest mountain in the alps. the alpine region area contains about a hundred peaks higher than 4, 000 m ( 13, 123 ft ), known as the " four - thousanders ". ho

In [24]:
len(y_train)

2

In [25]:
# `y_train[0]` and `y_train[1]` are the possitions in the text where the answer starts and ends, respectively.
y_train[0].shape, y_train[0].shape

((86136,), (86136,))

In [26]:
print('\n * CONTEXT:                   \n', train_squad_examples[sample].context)
print('\n * QUESTION:                  \n', train_squad_examples[sample].question)
print('\n * ANSWER (REFERENCE):        \n', train_squad_examples[sample].answer_text)

print('\n * ANSWER IN CONTEXT:         \n', tokenizer.decode(x_train[0][sample][ y_train[0][sample]:y_train[1][sample]+1 ]))

print('\n\n === TRAINING SAMPLE ===')
print('\n * CONTEXT & QUESTION:        \n', tokenizer.decode(x_train[0][sample]))
print('\n * POSITION IN CONTEXT:       \n', (y_train[0][sample], y_train[1][sample]))


 * CONTEXT:                   
 The Alps (/ælps/; Italian: Alpi [ˈalpi]; French: Alpes [alp]; German: Alpen [ˈʔalpm̩]; Slovene: Alpe [ˈáːlpɛ]) are the highest and most extensive mountain range system that lies entirely in Europe, stretching approximately 1,200 kilometres (750 mi) across eight Alpine countries: Austria, France, Germany, Italy, Liechtenstein, Monaco, Slovenia, and Switzerland. The Caucasus Mountains are higher, and the Urals longer, but both lie partly in Asia. The mountains were formed over tens of millions of years as the African and Eurasian tectonic plates collided. Extreme shortening caused by the event resulted in marine sedimentary rocks rising by thrusting and folding into high mountain peaks such as Mont Blanc and the Matterhorn. Mont Blanc spans the French–Italian border, and at 4,810 m (15,781 ft) is the highest mountain in the Alps. The Alpine region area contains about a hundred peaks higher than 4,000 m (13,123 ft), known as the "four-thousanders".

 * QUE